# changing the order of the picks so they will be read in correctly by NLLocLib.c


In [1]:
import os
import sys
import glob
import pandas as pd
import numpy as np
from obspy import read
from obspy.core import UTCDateTime

In [2]:
path='/Users/banjo/Projects/nll_pnsn/NLLoc/util'
ifile='/pnsn_in_CVM.hypo' 

In [3]:
datum = 0

In [4]:
# GOOD FOR ONE HYPO FILE
os.chdir(path)
ID=1
i=1
headers = []
PPicks = []
SPicks = []
MagPicks = []
evids = []
with open(path+ifile,'r') as fp:
    for line in fp.readlines():
        i=i+1
        if len(line) == 180:
            headers.append(line)
        elif len(line) == 121 or len(line)==120:            
            pha = line[14]
            if pha != ' ' and (line[11] == 'Z' or line[11] == '3'):
                PPicks.append([line])
            elif pha == 'P' and line[16] == '4':
                MagPicks.append(line)
            else:
                SPicks.append(line)
        else:
            evids.append(line)

In [5]:
# RUN THIS TO SORT A HYPOINVERSE FILE. 

# newHypo = np.concatenate([headers,PPicks,SPicks,MagPicks,evids])
# newHypo = np.concatenate([PPicks,SPicks])
# newHypo = str(newHypo)
# # bad_chars = ["\\n","'","[","]"]
# bad_chars = ["'[ ", "\\n", "']", "\\n","'","[","]"]


# for i in bad_chars:
#     newHypo = newHypo.replace(i,'')
    
# newHypo = newHypo.replace("\n ","\n")

# with open('/Users/banjo/Projects/nll_pnsn/PNSN_in_CVM_picks.out', 'w') as f:
#     f.write(newHypo.strip())
#     f.close()

## RUN EVERYTHING BELOW FOR A GTSRCE GENERATOR FOR THE FILE ABOVE. 

In [6]:
stns = []
for i in range(len(PPicks)):
    stn = PPicks[i][0][0:5]
    if stn[3] == " ":
        stn = PPicks[i][0][0:3]
        stns.append(stn)
    elif stn[4] ==" ":
        stn = PPicks[i][0][0:4]
        stns.append(stn)
    else:
        stn = PPicks[i][0][0:5]
        stns.append(stn)
#     stns.append(stn)

In [7]:
stas = np.unique(stns)

In [8]:
path2 = '/Users/banjo/Projects/nll_pnsn/util/stationInfo'
df1 = pd.read_csv(path2 + '/pnsnStations_simple.list')
# df2 = pd.read_csv(path + '/p4_stations_for_test.list')
# sta = df1['sta'].values
# lat = df1['lat'].values
# lon = df1['lon'].values
# ele = (df1['ele'].values)/1000

In [9]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))
# Driver Code
li1 = stas
li2 = np.unique(df1['sta'].values)
diff = Diff(li1, li2)
# print(diff)

In [10]:
match_sta = []
match_lat =[]
match_lon=[]
match_ele=[]
# tmp = df2['sta'].values
tmp = stas

for i in range(len(tmp)):
    station = tmp[i]
    if station in diff:
        pass
    else:
#         print(station)
        match = df1.loc[df1['sta'] == station].values[0]
#         print(i,match)
        match_lat.append(match[3])
        match_lon.append(match[4])
        match_ele.append(match[5])
        match_sta.append(match[1])

In [11]:
match_lat

[47.31964,
 47.21402,
 48.24313,
 47.45086,
 47.569179999999996,
 47.85586,
 45.61097,
 48.719590000000004,
 46.6855,
 47.5699,
 48.7197,
 48.7201,
 48.9983,
 49.0983,
 49.048,
 48.9534,
 48.9752,
 39.275936,
 38.43014,
 34.84845,
 39.138126,
 47.559528,
 38.438957,
 38.94596,
 39.039947999999995,
 38.79221,
 46.194790000000005,
 38.854839,
 38.567101,
 49.27,
 47.6469,
 45.81944,
 47.5751,
 47.320240000000005,
 38.929131,
 44.381816,
 46.10881,
 43.998160999999996,
 38.614719,
 46.09946,
 38.877029,
 46.6576,
 38.686256,
 38.764992,
 35.1269,
 43.82056,
 46.759879999999995,
 46.20067,
 38.823795000000004,
 46.152592,
 48.23617,
 45.73595,
 39.024307,
 46.14465,
 48.04307,
 48.06236,
 48.201925,
 48.059549,
 48.0588,
 48.057577,
 48.649543,
 48.924627,
 47.813,
 47.5133,
 46.996,
 46.9795,
 46.6527,
 46.3827,
 45.954609999999995,
 46.1112,
 45.6378,
 45.309351,
 44.493666999999995,
 43.9713,
 43.6643,
 43.668,
 43.2917,
 42.503514,
 42.505833,
 41.8308,
 48.0575,
 48.2641,
 48.2641,
 4

In [12]:
df = pd.DataFrame(list(zip(match_sta,match_lat,match_lon,match_ele)),columns = ['Station','Lat','Lon','Elev'])

In [13]:
df.drop_duplicates()

,Station,Lat,Lon,Elev
0,2193,47.319640,-122.58598,94
1,2194,47.214020,-123.10099,6
2,7026,48.243130,-122.45464,10
3,7030,47.450860,-122.30219,124
4,7034,47.569180,-122.62716,10
...,...,...,...,...
1376,YBH,41.731930,-122.71038,1059
1377,YEG,35.436580,-119.96009,907
1378,YEL,46.209550,-122.18899,1750
1379,YOCR,45.379178,-121.74354,1913


In [14]:
srcs = []
descriptor = []
zeros = []
lats = []
lons = []
elvs = []
names=[]
for i in range(len(match_lat)):
    lat = match_lat[i]
    lat = '%7.4f' %lat
    lon = match_lon[i]
    lon = '%7.4f' %lon
    ele = match_ele[i]/1000
    ele = '%6.2f' %ele
    name = match_sta[i]
    name = '%5s' %name
    name = str(name)
    srcs.append('GTSRCE')
    descriptor.append('LATLON')
    zeros.append('0.0')
    lats.append(lat)
    names.append(name)
    lons.append(lon)
    elvs.append(ele)

In [15]:
df = pd.DataFrame(list(zip(srcs,names,descriptor,lats,lons,zeros,elvs)),columns = ['Source','Station','Descriptor','Lat','Lon','Zeros','Elev'])

In [16]:
df = df.drop_duplicates()

In [18]:
df.to_csv('/Users/banjo/Projects/nll_pnsn/NLLoc/util/pnsn_in_CVM_gtsrce.list',index=0,header = False,sep =' ')

In [17]:
df

,Source,Station,Descriptor,Lat,Lon,Zeros,Elev
0,GTSRCE,2193,LATLON,47.3196,-122.5860,0.0,0.09
1,GTSRCE,2194,LATLON,47.2140,-123.1010,0.0,0.01
2,GTSRCE,7026,LATLON,48.2431,-122.4546,0.0,0.01
3,GTSRCE,7030,LATLON,47.4509,-122.3022,0.0,0.12
4,GTSRCE,7034,LATLON,47.5692,-122.6272,0.0,0.01
...,...,...,...,...,...,...,...
1376,GTSRCE,YBH,LATLON,41.7319,-122.7104,0.0,1.06
1377,GTSRCE,YEG,LATLON,35.4366,-119.9601,0.0,0.91
1378,GTSRCE,YEL,LATLON,46.2096,-122.1890,0.0,1.75
1379,GTSRCE,YOCR,LATLON,45.3792,-121.7435,0.0,1.91


In [19]:
diff

['GTS',
 'LCM',
 'LGP',
 'GWK',
 'GWR',
 'LRB',
 'GNA',
 'MCM',
 'KFP',
 'KCS',
 'LTC',
 'ABJ',
 'MEM',
 'SND',
 'LDB',
 'KIP',
 'GHM',
 'KJJ',
 'BOOK',
 'ORCA',
 'AOH',
 'KRK',
 'OGO',
 'COOM',
 'KSM',
 'BUT',
 'KPP',
 'LRR',
 'NBND',
 'GHL',
 'LMH',
 'HRY',
 'GAR',
 'KUHN',
 'LWH',
 'LRM',
 'OWY',
 'LBP',
 'LBK',
 'LME',
 'BRUIN',
 'SXM',
 'WIKI',
 'BGMT',
 'HBMT',
 'POC',
 'LSR',
 'GTWY',
 'LPG',
 'LRD',
 'CPI',
 'GHG',
 '7036',
 'DI2B',
 'AXEC3',
 'BBBB',
 '2172',
 'SOPS',
 'TPOR',
 'QPJE',
 'WIXX',
 'MOBC',
 'EAA',
 'H09A',
 '2161',
 'BNAB',
 'O02D',
 'LGOR',
 'GRUT',
 'PTD',
 'SNI2',
 '7028',
 'BDID',
 'AXBA1',
 'MID',
 'BLMT',
 'GUAC',
 'KEMF',
 'KCO',
 'SPK1',
 'EEA',
 'J10A',
 'UGAP3',
 'KCPB',
 '7037',
 'WEL1',
 'AXCC1',
 'MAYB',
 'H08A',
 '7033',
 'SFER',
 'IRG',
 'KMR',
 'KFO',
 'MYRA',
 'AXAS1',
 '2222',
 'NDB',
 'RCV',
 '7206',
 'HER',
 '7039',
 'DI2',
 'QEWU',
 'ECA',
 '7027',
 'G11A',
 'H3A',
 'L09A',
 'F09A',
 'EWN',
 'NCRB',
 'QDNP',
 'HLID',
 'I08A',
 'I10A',
 'HOLB'

In [20]:
len(diff)

274

In [ ]:
# WORKING ON MULTIPLE OBSERVATIONS IN ONE FILE VVVVV
os.chdir(path)
ID=1
i=1
events = []
headers = []
PPicks = []
SPicks = []
MagPicks = []
evids = []
with open(path+ifile,'r') as fp:
    for line in fp.readlines():
        if len(line) == 180:
            print(line)
            headers.append([line])
        elif:
            len(line) == 121 or len(line)==120   
#             print(line)
            pha = line[14]
            spha = line[47]
            if pha != ' ' and (line[11] == 'Z' or line[11] == '3'):
                PPicks.append(line)
            elif pha == 'P' and line[16] == '4':
                MagPicks.append(line)
            elif spha == 'S':
                SPicks.append(line)
        else:
            evids.append(line)
#         else:
            events.append([headers,PPicks,SPicks,MagPicks,evids])
#             print ("End line for eq. %s: %d'%(EVID,i-1)")

In [ ]:
events

In [ ]:
PPicks

In [ ]:
# newHypo = np.concatenate([headers,PPicks,SPicks,MagPicks,evids])
# newHypo = str(newHypo)
# # bad_chars = ["\\n","'","[","]"]
# bad_chars = ["'[ ", "\\n", "']", "\\n","'","[","]"]


# for i in bad_chars:
#     newHypo = newHypo.replace(i,'')
    
# newHypo = newHypo.replace("\n ","\n")

# with open('/Users/banjo/Projects/nll_pnsn/NLLoc/util/multiple_test.hypo', 'w') as f:
#     f.write(newHypo.strip())
#     f.close()

In [ ]:
bad_chars = ["\\n","'","[","]"]
# bad_chars = ["'[ ", "\\n", "']", "\\n","'","[","]"]
for i in bad_chars :
#     j = str(j)
    j = j.replace(i,'')

In [ ]:
j = j.strip()

In [ ]:
with open('/Users/banjo/Projects/nll_pnsn/test.hypo', 'w') as f:
    f.write(j)
    f.close()

In [ ]:
j = j.replace("\n ","\n")

In [ ]:
j

In [ ]:
# headers
PPicks

In [ ]:
os.chdir(path)
ID=1
i=1
column = ['event'] #,'TIME','LAT','LON','DEP','MAG','MTYPE','EH','EZ','RMS','EVID']
# stacol = ['NET','STA','CHAN']
eveinfo = pd.DataFrame(columns=column)
# nets = []
# stas = []
# chan =[]
stainfo = ['pick'] #,'ARR','ABSA','WEI','PHA','MOT','ON','DIR']
#     stainfo = ['STA','ARR','PHA']
# stainfo = ['EVID','STA','ARR','PHA']

tempsta = pd.DataFrame(columns = stainfo)
arrinfo = []
with open(path+ifile,'r') as fp:
    for line in fp.readlines():
        i=i+1
        if len(line) == 180:
#             year = line[0:4]
#             month = line[4:6]
#             day = line[6:8]
#             hour = line[8:10]
#             minu = line[10:12]
#             sec = line[12:16][:-2]+'.'+line[12:16][-2:]
#             ddir = year+'.'+month+'.'+day+'.'+hour+'.'+minu+'.'+line[12:16][:-2]
#             EVID= line[136:146].strip()
#             time = year+'-'+month+'-'+day+'T'+hour+':'+minu+':'+sec
#             time = UTCDateTime(time)
#             lat = float(line[16:18]) + float(line[19:23][:-2]+'.'+line[19:23][-2:].replace(' ','0'))/60.0
#             lon = -(float(line[23:26]) + float(line[27:31][:-2]+'.'+line[27:31][-2:].replace(' ','0'))/60.0)
#             dep = line[31:36]
#             if dep[-2]=='-':
#                 dep = dep[:-1]+'.0'+dep[-1:]
#             else:
#                 dep = (dep[:-2]+'.'+dep[-2:].replace(' ','0'))
#             dep = float(dep)+datum
#             rms = float((line[48:52][:-2]+'.'+line[48:52][-2:]).replace(' ','0'))
#             eh = float((line[85:89][:-2]+'.'+line[85:89][-2:]).replace(' ','0'))
#             ez = float((line[89:93][:-2]+'.'+line[89:93][-2:]).replace(' ','0'))
#             mc = line[70:73].strip()
#             mtype = line[122]
#             if len(mc)!= 0 and mc[0]=='-':
#                 mag = float(mc[:2]+'.'+mc[-1])
#             elif len(mc)!=0 and mc[0]!='-':
#                 mag = float(mc[:-2]+'.'+mc[-2:])
#             else:
#                 mag = -5 #coda magnitude
            newRow = {'event':line}
#             newRow = {'ID':int(ID),'TIME':time,'LAT':lat,'LON':lon,'DEP':dep,
#                       'MAG':mag,'MTYPE':mtype,'EH':eh,'EZ':ez,
#                       'RMS':rms,'EVID':EVID,'DIR':ddir}
            eveinfo = eveinfo.append(newRow,ignore_index=True)
            tempsta = tempsta.sort_values(by='pick')
            arrinfo.append(tempsta)
            tempsta = pd.DataFrame(columns = stainfo)
            ID=ID+1
        elif len(line) == 121 or len(line)==120: 
#             print(line)
            pha = line[14]
            spha = line[47]
            if pha != ' ' and (line[11] == 'Z' or line[11] == '3'):
                newStap = {'pick':line}
                tempsta = tempsta.append(newStap,ignore_index=True)
#                 PPicks.append(line)
            elif pha == 'P' and line[16] == '4':
                newStam = {'pick':line}
#                 MagPicks.append(line)
                tempsta = tempsta.append(newStam,ignore_index=True)
            elif spha == 'S':
                newStas = {'pick':line}
#                 SPicks.append(line)
                tempsta = tempsta.append(newStas,ignore_index=True)
            else:
                lastRow={'evid':line}
#             evids.append(line)
                tempsta = tempsta.append(lastRow,ignore_index=True)
            
            
#             sta = line[:5].strip()
#             if sta not in stas:
#                 stas.append(sta)
#                 nets.append(line[5:7].strip())
#                 chan.append(line[9:12])
            
#             #pull out the P phase
#             pha = line[14]
#             mot = line[15]
#             onset = line[13]
#             if pha != ' ' and (line[11] == 'Z' or line[11] == '3'):
#                 arryear = line[17:21]
#                 arrmonth = line[21:23]
#                 arrday = line[23:25]
#                 arrhour = line[25:27]
#                 arrmin = line[27:29]
#                 arrsec = (line[30:34][:-2]+'.'+line[30:34][-2:]).replace(' ','0')
#                 if float(arrsec) >= 60.0:
#                     arrsec = float(arrsec)-60.0
#                     arrtime = arryear+'-'+arrmonth+'-'+arrday+'T'+arrhour+':'+arrmin+':'+str(arrsec)
#                     arrtime = UTCDateTime(arrtime)+60.0
#                 else:
#                     arrtime = arryear+'-'+arrmonth+'-'+arrday+'T'+arrhour+':'+arrmin+':'+arrsec
#                     arrtime = UTCDateTime(arrtime)
#                 PTT = arrtime - time
#                 pweight = int(line[16])
#                 rpw = pweight
#                 newStap = {'pick':line} #,'STA':sta,'ARR':PTT,'PHA':pha}
#                     # newStap = {'STA':sta,'WEI':rpw,'ARR':PTT,'PHA':pha,'ABSA':arrtime,'MOT':mot,'ON':onset}
#                 tempsta = tempsta.append(newStap,ignore_index=True)

    
#             else:
#                  print ('Not avaliable P phase, not use in here')
#             spha = line[47]
#             if spha == 'S':
#                 arryear = line[17:21]
#                 arrmonth = line[21:23]
#                 arrday = line[23:25]
#                 arrhour = line[25:27]
#                 arrmin = line[27:29]
#                 arrsec = (line[42:46][:-2]+'.'+line[42:46][-2:]).replace(' ','0')
#                 if float(arrsec) >= 60:
#                     arrsec = float(arrsec)-60.0
#                     arrtime = arryear+'-'+arrmonth+'-'+arrday+'T'+arrhour+':'+arrmin+':'+str(arrsec)
#                     arrtime = UTCDateTime(arrtime)+60.0
#                 else:
#                     arrtime = arryear+'-'+arrmonth+'-'+arrday+'T'+arrhour+':'+arrmin+':'+arrsec
#                     arrtime = UTCDateTime(arrtime)
#                 STT = arrtime - time
#                 if STT < 0.0:
#                     print ("i-1")
#                     sys.exit('origin time %s and arrival time %s'%(time,arrtime))
#                 swei = int(line[49])
#                 rsw = swei
#                 newStas = {'EVID':EVID,'STA':sta,'ARR':STT,'PHA':spha}
#                     # newStas = {'STA':sta,'WEI':rsw,'ARR':STT,'PHA':spha,'ABSA':arrtime}
#                 tempsta = tempsta.append(newStas,ignore_index=True)

#         else:
#             print ("End line for eq. %s: %d'%(EVID,i-1)")

arrinfo.append(tempsta)
arrinfo.pop(0)
# arrdf = pd.DataFrame(arrinfo)
eveinfo['pick']=arrinfo
# stainfos = pd.DataFrame(np.transpose([nets,stas,chan]),columns=stacol)

# print(stainfos)
# print(arrinfo)

In [ ]:
evid_1 = [arrinfo[0]]

In [ ]:
evid_1

In [ ]:
print(eveinfo.event[0], eveinfo.pick[0])

In [ ]:
print(str(eveinfo.pick[0]))

In [ ]:
os.chdir(path)
ID=1
i=1
column = ['ID','TIME','LAT','LON','DEP','MAG','MTYPE','EH','EZ','RMS','EVID']
stacol = ['NET','STA','CHAN']
eveinfo = pd.DataFrame(columns=column)
nets = []
stas = []
chan =[]
stainfo = ['STA','ARR','ABSA','WEI','PHA','MOT','ON','DIR']
#     stainfo = ['STA','ARR','PHA']
# stainfo = ['EVID','STA','ARR','PHA']

tempsta = pd.DataFrame(columns = stainfo)
arrinfo = []
with open(path+ifile,'r') as fp:
    for line in fp.readlines():
        i=i+1
        if len(line) == 180:
            year = line[0:4]
            month = line[4:6]
            day = line[6:8]
            hour = line[8:10]
            minu = line[10:12]
            sec = line[12:16][:-2]+'.'+line[12:16][-2:]
            ddir = year+'.'+month+'.'+day+'.'+hour+'.'+minu+'.'+line[12:16][:-2]
            EVID= line[136:146].strip()
            time = year+'-'+month+'-'+day+'T'+hour+':'+minu+':'+sec
            time = UTCDateTime(time)
            lat = float(line[16:18]) + float(line[19:23][:-2]+'.'+line[19:23][-2:].replace(' ','0'))/60.0
            lon = -(float(line[23:26]) + float(line[27:31][:-2]+'.'+line[27:31][-2:].replace(' ','0'))/60.0)
            dep = line[31:36]
            if dep[-2]=='-':
                dep = dep[:-1]+'.0'+dep[-1:]
            else:
                dep = (dep[:-2]+'.'+dep[-2:].replace(' ','0'))
            dep = float(dep)+datum
            rms = float((line[48:52][:-2]+'.'+line[48:52][-2:]).replace(' ','0'))
            eh = float((line[85:89][:-2]+'.'+line[85:89][-2:]).replace(' ','0'))
            ez = float((line[89:93][:-2]+'.'+line[89:93][-2:]).replace(' ','0'))
            mc = line[70:73].strip()
            mtype = line[122]
            if len(mc)!= 0 and mc[0]=='-':
                mag = float(mc[:2]+'.'+mc[-1])
            elif len(mc)!=0 and mc[0]!='-':
                mag = float(mc[:-2]+'.'+mc[-2:])
            else:
                mag = -5 #coda magnitude
            newRow = {'ID':int(ID),'TIME':time,'LAT':lat,'LON':lon,'DEP':dep,
                      'MAG':mag,'MTYPE':mtype,'EH':eh,'EZ':ez,
                      'RMS':rms,'EVID':EVID,'DIR':ddir}
            eveinfo = eveinfo.append(newRow,ignore_index=True)
            tempsta = tempsta.sort_values(by='ARR')
            arrinfo.append(tempsta)
            tempsta = pd.DataFrame(columns = stainfo)
            ID=ID+1
        elif len(line) == 121 or len(line)==120:
            sta = line[:5].strip()
            if sta not in stas:
                stas.append(sta)
                nets.append(line[5:7].strip())
                chan.append(line[9:12])
            
            #pull out the P phase
            pha = line[14]
            mot = line[15]
            onset = line[13]
            if pha != ' ' and (line[11] == 'Z' or line[11] == '3'):
                arryear = line[17:21]
                arrmonth = line[21:23]
                arrday = line[23:25]
                arrhour = line[25:27]
                arrmin = line[27:29]
                arrsec = (line[30:34][:-2]+'.'+line[30:34][-2:]).replace(' ','0')
                if float(arrsec) >= 60.0:
                    arrsec = float(arrsec)-60.0
                    arrtime = arryear+'-'+arrmonth+'-'+arrday+'T'+arrhour+':'+arrmin+':'+str(arrsec)
                    arrtime = UTCDateTime(arrtime)+60.0
                else:
                    arrtime = arryear+'-'+arrmonth+'-'+arrday+'T'+arrhour+':'+arrmin+':'+arrsec
                    arrtime = UTCDateTime(arrtime)
                PTT = arrtime - time
                pweight = int(line[16])
                rpw = pweight
                newStap = {'EVID':EVID,'STA':sta,'ARR':PTT,'PHA':pha}
                    # newStap = {'STA':sta,'WEI':rpw,'ARR':PTT,'PHA':pha,'ABSA':arrtime,'MOT':mot,'ON':onset}
                tempsta = tempsta.append(newStap,ignore_index=True)

    
            else:
                 print ('Not avaliable P phase, not use in here')
            spha = line[47]
            if spha == 'S':
                arryear = line[17:21]
                arrmonth = line[21:23]
                arrday = line[23:25]
                arrhour = line[25:27]
                arrmin = line[27:29]
                arrsec = (line[42:46][:-2]+'.'+line[42:46][-2:]).replace(' ','0')
                if float(arrsec) >= 60:
                    arrsec = float(arrsec)-60.0
                    arrtime = arryear+'-'+arrmonth+'-'+arrday+'T'+arrhour+':'+arrmin+':'+str(arrsec)
                    arrtime = UTCDateTime(arrtime)+60.0
                else:
                    arrtime = arryear+'-'+arrmonth+'-'+arrday+'T'+arrhour+':'+arrmin+':'+arrsec
                    arrtime = UTCDateTime(arrtime)
                STT = arrtime - time
                if STT < 0.0:
                    print ("i-1")
                    sys.exit('origin time %s and arrival time %s'%(time,arrtime))
                swei = int(line[49])
                rsw = swei
                newStas = {'EVID':EVID,'STA':sta,'ARR':STT,'PHA':spha}
                    # newStas = {'STA':sta,'WEI':rsw,'ARR':STT,'PHA':spha,'ABSA':arrtime}
                tempsta = tempsta.append(newStas,ignore_index=True)

        else:
            print ("End line for eq. %s: %d'%(EVID,i-1)")

arrinfo.append(tempsta)

arrinfo.pop(0)
# arrdf = pd.DataFrame(arrinfo)
eveinfo['ARR']=arrinfo
#     test = 
stainfos = pd.DataFrame(np.transpose([nets,stas,chan]),columns=stacol)

# print(stainfos)
# print(arrinfo)

In [ ]:
arrinfo

In [ ]:
eveinfo

In [ ]:
eveinfo.ARR[0]

In [ ]:
# stainfos.to_csv(stn_file,sep = ' ' ,index = 0)

In [ ]:
eveinfo

In [ ]:
arrinfo

In [ ]:
# arrinfo.to_csv('p4_test_phaseArrivals.csv', index = 0)

In [ ]:
ard = np.array(arrinfo)
ard1 = ard[0]
print(len(ard1))

In [ ]:
stns = []
tts= []
phas=[]
for i in range(len(ard1)):
    station = ard1[i][1]
    tt = ard1[i][2]
    pha = ard1[i][3]
    stns.append(station)
    tts.append(tt)
    phas.append(pha)

In [ ]:
dfa = pd.DataFrame(list(zip(stns,tts,phas)),columns = ['sta','arr','pha'])

In [ ]:
dfa.sort_values('pha')

In [ ]:
stainfos

In [ ]:
stations = np.unique(stainfos['STA'].values)

In [ ]:
df2 = pd.DataFrame(stations, columns =['stn'])

In [ ]:
# for i in range(len(stations)):
#     sta = stations[i]
#     stainfo 
#     print(sta)

In [ ]:
path2 = '/Users/banjo/Projects/nll_pnsn/util/stationInfo'
df1 = pd.read_csv(path2 + '/pnsnStations_simple.list')
# df2 = pd.read_csv(path + '/p4_stations_for_test.list')


In [ ]:
match_sta = []
match_lat =[]
match_lon=[]
match_ele=[]
tmp = df2['stn'].values
for i in range(len(tmp)):
    station = tmp[i]
    match = df1.loc[df1['sta']==station].values[0]
    match_lat.append(match[3])
    match_lon.append(match[4])
    match_ele.append(match[5])
    match_sta.append(match[1])

In [ ]:
df = pd.DataFrame(list(zip(match_sta,match_lat,match_lon,match_ele)),columns = ['Station','Lat','Lon','Elev'])

In [ ]:
df.drop_duplicates()

In [ ]:
srcs = []
descriptor = []
zeros = []
lats = []
lons = []
elvs = []
names=[]
for i in range(len(match_lat)):
    lat = match_lat[i]
    lat = '%5.2f' %lat
    lon = match_lon[i]
    lon = '%5.2f' %lon
    ele = match_ele[i]/1000
    ele = '%6.2f' %ele
    name = match_sta[i]
    name = '%5s' %name
    name = str(name)
    srcs.append('GTSRCE')
    descriptor.append('LATLON')
    zeros.append('0.0')
    lats.append(lat)
    names.append(name)
    lons.append(lon)
    elvs.append(ele)

In [ ]:
df = pd.DataFrame(list(zip(srcs,names,descriptor,lats,lons,zeros,elvs)),columns = ['Source','Station','Descriptor','Lat','Lon','Zeros','Elev'])

In [ ]:
df = df.drop_duplicates()

In [ ]:
# df

In [ ]:
df.to_csv(g2t_stn_list4plot,index=0)

In [ ]:
df

In [ ]:
df.to_csv(g2t_stn_list,sep =' ',index=0,header=False)

In [ ]:
# df